In [1]:
debug = 1
import numpy as np
import pandas
pandas.set_option('display.max_colwidth', -1)
states = ['ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc']
states += ['de', 'fl', 'ga', 'hi', 'ia', 'id', 'il', 'in']
states += ['ks', 'ky', 'la', 'ma', 'md', 'me', 'mi', 'mn']
states += ['mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj']
states += ['nh', 'nj', 'nm', 'nv', 'ny', 'oh', 'ok', 'or']
states += ['pa', 'pr', 'ri', 'sc', 'sd', 'tn', 'tx', 'us']
states += ['ut' ,'va', 'vt', 'wa', 'wi', 'wv', 'wy']

data_df = pandas.DataFrame()
metadata_df = pandas.DataFrame()

for i in range(1,55):
    if (debug >= 1):
        print('Importing data for sequence {0:,.0f}...'.format(i))
    this_seq_data_df = pandas.DataFrame()
    this_seq_metadata_filename = 'metadata/Seq{0:.0f}.xls'.format(i)
    this_seq_metadata_df = pandas.read_excel(this_seq_metadata_filename, header=None)
    
    this_seq_metadata_df = this_seq_metadata_df.dropna(axis=1)
    this_seq_metadata_df.columns = this_seq_metadata_df.loc[0]
    if (i == 1):
        this_seq_metadata_df.columns = [['FILEID','FILETYPE','STUSAB','CHARITER','SEQUENCE','LOGRECNO','A00001_001','A00001_002']]
    this_seq_metadata_df = this_seq_metadata_df.drop(0, axis=0)
    this_seq_metadata_df.index = ['description']
    
    for onestate in states:
        statefilename = 'data/seq{0:.0f}/e20161{1:s}{2:04d}000.txt'.format(i,onestate,i)
        onestate_df = pandas.read_csv(statefilename, header=None)
        this_seq_data_df = this_seq_data_df.append(onestate_df)

    this_seq_data_df.columns = this_seq_metadata_df.columns
    
    if (i >= 2):
        this_seq_metadata_df = this_seq_metadata_df.drop(['FILEID','FILETYPE','STUSAB','CHARITER','SEQUENCE','LOGRECNO'], axis=1)
        this_seq_data_df = this_seq_data_df.drop(['FILEID','FILETYPE','STUSAB','CHARITER','SEQUENCE','LOGRECNO'], axis=1)

    if (debug >= 2):
        print('Merging datasets...')
    metadata_df = pandas.concat((metadata_df, this_seq_metadata_df), axis=1)
    data_df = pandas.concat((data_df, this_seq_data_df), axis=1)
        
if (debug >= 2):
    print('Creating pseudo geoid...')
data_df['PSEUDO_GEOID'] = data_df['STUSAB'].str.upper() + data_df['LOGRECNO'].apply(lambda x: '{0:07d}'.format(x))
metadata_df['PSEUDO_GEOID'] = 'Temporary ID constructed from the STUSAB and the 7-digit zero-padded LOGRECNO to use to look up the geoid later'

if (debug >= 2):
    print('Indexing...')
data_df = data_df.set_index('PSEUDO_GEOID')
metadata_df = metadata_df.set_index('PSEUDO_GEOID')

if (debug >= 2):
    print('Reading geography file...')
geoid_df = pandas.read_excel('metadata/geoid.xlsx')
geoid_df = geoid_df.set_index('PSEUDO_GEOID')

if (debug >= 2):
    print('Merging geography with data...')
data_df = data_df.join(geoid_df[['GEOID','NAME']])
#data_df = data_df.rename(columns={'NAME': 'GEO_NAME'})

metadata_df['GEOID'] = 'Geography identifier'
metadata_df['NAME'] = 'Name of geography unit'

if (debug >= 2):
    print('Resetting index...')
data_df = data_df.set_index('GEOID')
metadata_df = metadata_df.set_index('GEOID')

if (debug >= 2):
    print('Rearranging columns')
cols = data_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
data_df = data_df[cols]
metadata_df = metadata_df[cols]
metadata_df = metadata_df.T
metadata_df = metadata_df.reset_index()
metadata_df.columns = ['variable', 'description']
metadata_df = metadata_df.set_index('variable', drop=False)

if (debug >= 1):
    print('Writing variables file...')
metadata_df.to_csv('variables_acs2016_1_54.csv')

if (debug >= 1):
    print('Writing data (estimates) file...')
data_df.to_csv('data_acs2016_1_54.csv')

print('Done!')

Importing data for sequence 1...
Importing data for sequence 2...
Importing data for sequence 3...
Importing data for sequence 4...
Importing data for sequence 5...
Importing data for sequence 6...
Importing data for sequence 7...
Importing data for sequence 8...
Importing data for sequence 9...
Importing data for sequence 10...
Importing data for sequence 11...
Importing data for sequence 12...
Importing data for sequence 13...
Importing data for sequence 14...
Importing data for sequence 15...
Importing data for sequence 16...
Importing data for sequence 17...
Importing data for sequence 18...
Importing data for sequence 19...
Importing data for sequence 20...
Importing data for sequence 21...
Importing data for sequence 22...
Importing data for sequence 23...
Importing data for sequence 24...
Importing data for sequence 25...
Importing data for sequence 26...
Importing data for sequence 27...
Importing data for sequence 28...
Importing data for sequence 29...
Importing data for sequ

In [ ]:
varlist = []
string1 = 'Total Population'
string2 = 'AGE'
string3 = 'Under 5'

#string1 = 'TRAVEL TIME
#string2 = '10 to 14 Minutes
#string3 = 'bus'
#string1 = 'MALE'
#string2 = ' '
#string3 = ' '
#string2 = 'all'
#string3 = ''

varlist = metadata_df['variable'][metadata_df['description'].apply(lambda x: (string1 in x) & (string2 in x) & (string3 in x))].values.tolist()


for thisvar in varlist:
    print(thisvar)
    print(metadata_df['description'].loc[thisvar])
    print('\n')

#data_df['B01001_002'].head(2)

In [2]:
data_df

,NAME,FILEID,FILETYPE,STUSAB,CHARITER,SEQUENCE,LOGRECNO,A00001_001,A00001_002,B01001_001,...,C14007H_005,C14007H_006,C14007H_007,C14007I_001,C14007I_002,C14007I_003,C14007I_004,C14007I_005,C14007I_006,C14007I_007
GEOID,,,,,,,,,,,,,,,,,,,,,
04000US02,Alaska,ACSSF,20160.0,ak,0,1,1,18493,8528,741894.0,...,20123.0,27349.0,339518.0,47770.0,17036.0,1833.0,7539.0,4022.0,3642.0,30734.0
04001US02,Alaska -- Urban,ACSSF,20160.0,ak,0,1,2,7618,2943,482038.0,...,12904.0,20792.0,215893.0,35296.0,13333.0,1517.0,5343.0,3276.0,3197.0,21963.0
04043US02,Alaska -- Rural,ACSSF,20160.0,ak,0,1,3,10875,5585,259856.0,...,7219.0,6557.0,123625.0,12474.0,3703.0,316.0,2196.0,746.0,445.0,8771.0
040A0US02,Alaska -- In metropolitan or micropolitan statistical area,ACSSF,20160.0,ak,0,1,4,8630,3692,549167.0,...,15882.0,24059.0,265220.0,39414.0,14519.0,1542.0,6136.0,3462.0,3379.0,24895.0
040C0US02,Alaska -- In metropolitan statistical area,ACSSF,20160.0,ak,0,1,5,7579,3229,503162.0,...,14594.0,22921.0,241494.0,36775.0,13368.0,1454.0,5555.0,3358.0,3001.0,23407.0
040C1US02,Alaska -- In metropolitan statistical area -- in principal city,ACSSF,20160.0,ak,0,1,6,4472,1679,330948.0,...,8263.0,14819.0,144848.0,28033.0,10124.0,1215.0,3673.0,2887.0,2349.0,17909.0
040C2US02,Alaska -- In metropolitan statistical area -- not in principal city,ACSSF,20160.0,ak,0,1,7,3107,1550,172214.0,...,6331.0,8102.0,96646.0,8742.0,3244.0,239.0,1882.0,471.0,652.0,5498.0
040G0US02,Alaska -- Not in metropolitan or micropolitan statistical area,ACSSF,20160.0,ak,0,1,8,9863,4836,192727.0,...,4241.0,3290.0,74298.0,8356.0,2517.0,291.0,1403.0,560.0,263.0,5839.0
040H0US02,Alaska -- Not in metropolitan statistical area,ACSSF,20160.0,ak,0,1,9,10914,5299,238732.0,...,5529.0,4428.0,98024.0,10995.0,3668.0,379.0,1984.0,664.0,641.0,7327.0


In [3]:
debug = 1
import numpy as np
import pandas
states = ['ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc']
states += ['de', 'fl', 'ga', 'hi', 'ia', 'id', 'il', 'in']
states += ['ks', 'ky', 'la', 'ma', 'md', 'me', 'mi', 'mn']
states += ['mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj']
states += ['nh', 'nj', 'nm', 'nv', 'ny', 'oh', 'ok', 'or']
states += ['pa', 'pr', 'ri', 'sc', 'sd', 'tn', 'tx', 'us']
states += ['ut' ,'va', 'vt', 'wa', 'wi', 'wv', 'wy']

error_df = pandas.DataFrame()
metadata_df = pandas.DataFrame()

for i in range(1,55):
    if (debug >= 1):
        print('Importing data for sequence {0:,.0f}...'.format(i))
    this_seq_error_df = pandas.DataFrame()
    this_seq_metadata_filename = 'metadata/Seq{0:.0f}.xls'.format(i)
    this_seq_metadata_df = pandas.read_excel(this_seq_metadata_filename, header=None)
    
    this_seq_metadata_df = this_seq_metadata_df.dropna(axis=1)
    this_seq_metadata_df.columns = this_seq_metadata_df.loc[0]
    if (i == 1):
        this_seq_metadata_df.columns = [['FILEID','FILETYPE','STUSAB','CHARITER','SEQUENCE','LOGRECNO','A00001_001','A00001_002']]
    this_seq_metadata_df = this_seq_metadata_df.drop(0, axis=0)
    this_seq_metadata_df.index = ['description']
    
    for onestate in states:
        statefilename = 'data/seq{0:.0f}/m20161{1:s}{2:04d}000.txt'.format(i,onestate,i)
        onestate_df = pandas.read_csv(statefilename, header=None)
        this_seq_error_df = this_seq_error_df.append(onestate_df)

    this_seq_error_df.columns = this_seq_metadata_df.columns
    
    if (i >= 2):
        this_seq_metadata_df = this_seq_metadata_df.drop(['FILEID','FILETYPE','STUSAB','CHARITER','SEQUENCE','LOGRECNO'], axis=1)
        this_seq_error_df = this_seq_error_df.drop(['FILEID','FILETYPE','STUSAB','CHARITER','SEQUENCE','LOGRECNO'], axis=1)

    if (debug >= 2):
        print('Merging datasets...')
    metadata_df = pandas.concat((metadata_df, this_seq_metadata_df), axis=1)
    error_df = pandas.concat((error_df, this_seq_error_df), axis=1)
        
if (debug >= 2):
    print('Creating pseudo geoid...')
error_df['PSEUDO_GEOID'] = error_df['STUSAB'].str.upper() + error_df['LOGRECNO'].apply(lambda x: '{0:07d}'.format(x))
metadata_df['PSEUDO_GEOID'] = 'Temporary ID constructed from the STUSAB and the 7-digit zero-padded LOGRECNO to use to look up the geoid later'

if (debug >= 2):
    print('Indexing...')
error_df = error_df.set_index('PSEUDO_GEOID')
metadata_df = metadata_df.set_index('PSEUDO_GEOID')

if (debug >= 2):
    print('Reading geography file...')
geoid_df = pandas.read_excel('metadata/geoid.xlsx')
geoid_df = geoid_df.set_index('PSEUDO_GEOID')

if (debug >= 2):
    print('Merging geography with data...')
error_df = error_df.join(geoid_df[['GEOID','NAME']])
#error_df = error_df.rename(columns={'NAME': 'GEO_NAME'})

metadata_df['GEOID'] = 'Geography identifier'
metadata_df['NAME'] = 'Name of geography unit'

if (debug >= 2):
    print('Resetting index...')
error_df = error_df.set_index('GEOID')
metadata_df = metadata_df.set_index('GEOID')

if (debug >= 2):
    print('Rearranging columns')
cols = error_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
error_df = error_df[cols]
metadata_df = metadata_df[cols]
metadata_df = metadata_df.T
metadata_df = metadata_df.reset_index()
metadata_df.columns = ['variable', 'description']
metadata_df = metadata_df.set_index('variable', drop=False)

#print('Writing variables file...')
#metadata_df.to_csv('variables_acs2016_1_30.csv')

if (debug >= 1):
    print('Writing margins of error file...')
error_df.to_csv('error_acs2016_1_54.csv')

print('Done!')

#error_df.sample(4)

Importing data for sequence 1...
Importing data for sequence 2...
Importing data for sequence 3...
Importing data for sequence 4...
Importing data for sequence 5...
Importing data for sequence 6...
Importing data for sequence 7...
Importing data for sequence 8...
Importing data for sequence 9...
Importing data for sequence 10...
Importing data for sequence 11...
Importing data for sequence 12...
Importing data for sequence 13...
Importing data for sequence 14...
Importing data for sequence 15...
Importing data for sequence 16...
Importing data for sequence 17...
Importing data for sequence 18...
Importing data for sequence 19...
Importing data for sequence 20...
Importing data for sequence 21...
Importing data for sequence 22...
Importing data for sequence 23...
Importing data for sequence 24...
Importing data for sequence 25...
Importing data for sequence 26...
Importing data for sequence 27...
Importing data for sequence 28...
Importing data for sequence 29...
Importing data for sequ

In [ ]:
import numpy as np
import pandas
states = ['ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc']
states += ['de', 'fl', 'ga', 'hi', 'ia', 'id', 'il', 'in']
states += ['ks', 'ky', 'la', 'ma', 'md', 'me', 'mi', 'mn']
states += ['mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj']
states += ['nh', 'nj', 'nm', 'nv', 'ny', 'oh', 'ok', 'or']
states += ['pa', 'pr', 'ri', 'sc', 'sd', 'tn', 'tx', 'us']
states += ['ut' ,'va', 'vt', 'wa', 'wi', 'wv', 'wy']

geo_df = pandas.DataFrame()
geo_metadata_df = pandas.DataFrame()

print('Importing geography data...')

geo_metadata_filename = 'metadata/2016_SFGeoFileTemplate.xls'
geo_metadata_df = pandas.read_excel(geo_metadata_filename, header=None)

geo_metadata_df = geo_metadata_df.dropna(axis=1)
geo_metadata_df.columns = geo_metadata_df.loc[0]
geo_metadata_df = geo_metadata_df.drop(0, axis=0)
geo_metadata_df.index = ['description']

for onestate in states:
    statefilename = 'geography/g20161{0:}.csv'.format(onestate)
    #print(statefilename)
    onestate_df = pandas.read_csv(statefilename, header=None, encoding='utf-8')
    geo_df = geo_df.append(onestate_df)

geo_df.columns = geo_metadata_df.columns
        
geo_df = geo_df.set_index('GEOID')

print('Writing geography variables file...')
geo_metadata_df.to_csv('geo_variables_acs2016.csv')

print('Writing geography file')
geo_df.to_csv('geo_acs2016.csv')

print('Done!')

In [ ]:
varlist = []
string1 = 'Baltimore'
#string2 = 'Male'
#string3 = ' '

#string1 = 'TRAVEL TIME
#string2 = '10 to 14 Minutes
#string3 = 'bus'
#string1 = 'MALE'
string2 = ' '
string3 = ' '
#string2 = 'all'
#string3 = ''

geo_df['NAME'][geo_df['NAME'].apply(lambda x: (string1 in x) & (string2 in x) & (string3 in x))]

In [ ]:
error_df['B01001_027'].loc['05000US24510']
#data_df